In [1]:
!pip install selenium
!pip install webdriver-manager
!pip-get update

'pip-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [3]:
import requests

import time
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

import pandas as pd

In [165]:
# url
store_id = '1735144615'
url = 'https://m.place.naver.com/restaurant/' + store_id + '/home?entry=plt&reviewSort=recent'

In [167]:
header_info = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, likeGecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'}

In [251]:
# 옵션 생성
options = webdriver.ChromeOptions()

# 옵션 추가
options.add_argument('--headless') # GUI 없이 브라우저가 구동되도록
options.add_argument('--no-sandbox') # 보안 문제를 피할 수 있도록
options.add_argument("--lang=ko") # 언어변경
options.add_argument('disable-gpu') # GPU를 사용하지 않도록 설정

driver = webdriver.Chrome() 
driver.maximize_window() # 크롬창 크기 maximize로 조절
time.sleep(3)
driver.get(url)

In [325]:
def get_store_info(store_id):
    url = 'https://m.place.naver.com/restaurant/' + store_id + '/home?entry=plt&reviewSort=recent'

    store_info = dict()
    
    # Start crawling/scraping!
    try:
        driver = webdriver.Chrome()
        res = driver.get(url)
        driver.implicitly_wait(30)
        

        # 주소 추출
        time.sleep(1)
        try:
            address = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/a/span[1]')
            store_info['address'] = address.text
            print("가게 주소:", store_info['address'])
        except Exception:
            store_info['address'] = ''
            print("가게 주소: 없음")

        # 영업시간 추출
        time.sleep(1)
        try:
            button = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div[1]/div/div[2]/div/a/div/div')
            button.click()
            # Pagedown
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(1)
            store_time = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div[1]/div/div[2]/div')
            store_info['store_time'] = store_time.text
            print("가게 영업시간:", store_info['store_time'])
        except Exception:
            store_info['store_time'] = ''
            print("가게 영업시간: 없음")

        # 가게 정보 추출
        time.sleep(1)
        try:
            info = driver.find_element(By.CSS_SELECTOR, 'div.vV_z_ > div.xPvPE')
            store_info['info'] = info.text
            print("가게 세부 정보:", store_info['info'])
        except Exception:
            store_info['info'] = ''
            print("가게 세부 정보: 없음")
        

        # 결과 반환
        return store_info

    except Exception as e:
        print(e)
        return store_info

In [299]:
store_id = '1735144615'
store_info = get_store_info(store_id)

가게 주소: 서울 서초구 남부순환로356길 18-5 2층 이층완우
가게 영업시간: 브레이크타임
17:30에 영업 시작
17시 30분에 영업 시작
수
11:30 - 23:00
14:00 - 17:30 브레이크타임
22:00 라스트오더
목
11:30 - 23:00
14:00 - 17:30 브레이크타임
22:00 라스트오더
금
11:30 - 23:00
14:00 - 17:30 브레이크타임
22:00 라스트오더
토
17:00 - 23:00
22:00 라스트오더
일
정기휴무 (매주 일요일)
월
11:30 - 23:00
14:00 - 17:30 브레이크타임
22:00 라스트오더
화
11:30 - 23:00
14:00 - 17:30 브레이크타임
22:00 라스트오더
- 마감시간은 22:30~23시 유동적으로 운영합니다.
접기
가게 세부 정보: 예약, 무선 인터넷


In [301]:
store_info

{'address': '서울 서초구 남부순환로356길 18-5 2층 이층완우',
 'store_time': '브레이크타임\n17:30에 영업 시작\n17시 30분에 영업 시작\n수\n11:30 - 23:00\n14:00 - 17:30 브레이크타임\n22:00 라스트오더\n목\n11:30 - 23:00\n14:00 - 17:30 브레이크타임\n22:00 라스트오더\n금\n11:30 - 23:00\n14:00 - 17:30 브레이크타임\n22:00 라스트오더\n토\n17:00 - 23:00\n22:00 라스트오더\n일\n정기휴무 (매주 일요일)\n월\n11:30 - 23:00\n14:00 - 17:30 브레이크타임\n22:00 라스트오더\n화\n11:30 - 23:00\n14:00 - 17:30 브레이크타임\n22:00 라스트오더\n- 마감시간은 22:30~23시 유동적으로 운영합니다.\n접기',
 'info': '예약, 무선 인터넷'}

In [327]:
def multiple_store_info(store_ids):
    store_info_dict = []  # 모든 가게 데이터를 저장할 리스트

    for store_id in store_ids:
        print(f"Get store info: {store_id}")
        store_info = get_store_info(store_id)
        
        # 가게 정보를 딕셔너리로 저장
        store_info = {
            "store_id": store_id,
            "store_info": store_info
        }
        store_info_dict.append(store_info)

    return store_info_dict

In [331]:
store_ids = ['1582818366', '37233088', '38264349', '1113983640', '32254662', '1427943416', '1879800911', '209145847', '1838844768', '1106978299', '18747564', '964119437', '12981147', '46696494', '38229207', '179368379', '1188227652', '32708325', '38248908', '12962167', '1735144615', '193891932', '1906380511', '1360440438', '74715630', '13151847', '1660867179', '151264987', '35136252', '1257299870', '12917286', '573699969', '620225564', '38709080', '1419339898', '36122875', '1090964067', '765084210', '11680229', '1618656208', '13160546', '1429930181', '37657812', '34521207', '13150657', '13323748', '1173026949', '1417896057', '38251615', '1123021012', '34345158', '37889898', '13376678', '1830398009', '37307613', '1927679596']

print(len(store_ids))

56


In [337]:
store_ids = ['1886378828', '1891561259', '1278613093', '13348519', '1567180996', '31186755', '1807443956', '1944962798', '31600260', '234208683', '38474259', '1672141709', '1658172503', '1037966057', '1786476142', '36538658', '20897027', '37615410', '35155951', '1034622849', '30922907', '20601502', '1889974700', '1672897084', '35307516', '34098856']

print(len(store_ids))

26


In [343]:
store_ids = ['12950069', '1664070278', '1154400830', '33459979', '32699962', '1069491314', '19882315', '1081786133', '1625687301', '38643869', '36868411', '20874669', '13157782', '1192671730', '38591616', '1320179295', '11861500', '1078767514', '31134978', '18771333', '37997726', '37236849', '1268565080', '37902481', '32821881', '1044461398', '11627634', '18779141', '98898756', '13150366', '1550997885', '34582729', '18361460', '1532111915', '1925656389', '20083769', '1137666363', '1553486347', '1688755338', '1516711006', '1932467805', '1698302178', '1113983640', '1751873950', '18307489', '1941253545', '1827894067', '13150440', '1361137881', '1420055829', '1931613846', '36200001', '13150444']

print(len(store_ids))

53


In [345]:
# 가게별 크롤링 실행
all_store_info = multiple_store_info(store_ids)

# 크롤링 결과 출력
for store_info in all_store_info:
    print(store_info)

Get store info: 12950069
가게 주소: 서울 서초구 강남대로 213 엘타워 4층
가게 영업시간: 브레이크타임
18:00에 영업 시작
18시 0분에 영업 시작
수
12:00 - 21:00
14:30 - 18:00 브레이크타임
목
12:00 - 21:00
14:30 - 18:00 브레이크타임
금
12:00 - 21:00
14:30 - 18:00 브레이크타임
토
12:00 - 21:30
14:30 - 17:00 브레이크타임
19:00 - 19:30 브레이크타임
일
12:00 - 20:30
14:30 - 17:30 브레이크타임
월
12:00 - 21:00
14:30 - 18:00 브레이크타임
화
12:00 - 21:00
14:30 - 18:00 브레이크타임
- 오전10시부터 전화예약가능 / 토요일만 2부제운영
접기
가게 세부 정보: 예약, 단체 이용 가능, 남/녀 화장실 구분, 포장, 배달, 무선 인터넷, 주차, 발렛파킹
Get store info: 1664070278
가게 주소: 서울 서초구 강남대로 221
가게 영업시간: 없음
가게 세부 정보: 없음
Get store info: 1154400830
가게 주소: 서울 서초구 강남대로 243
가게 영업시간: 없음
가게 세부 정보: 없음
Get store info: 33459979
가게 주소: 서울 서초구 남부순환로 2579 1층 치킨처럼
가게 영업시간: 영업 중
00:30에 영업 종료
0시 30분에 영업 종료
수
15:00 - 00:30
목
15:00 - 00:30
금
15:00 - 00:30
토
15:00 - 24:00
일
정기휴무 (매주 일요일)
월
15:00 - 00:30
화
15:00 - 00:30
접기
가게 세부 정보: 단체 이용 가능, 포장, 무선 인터넷, 예약, 남/녀 화장실 구분, 주차
Get store info: 32699962
가게 주소: 서울 서초구 남부순환로 2581-5 대호빌딩
가게 영업시간: 영업 중
22:30에 영업 종료
22시 30분에 영업 종료
매일
11:00 - 22:

In [347]:
import json

def save_data_to_json(all_store_info, file_name = 'store_info_db3.json'):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(all_store_info, f, ensure_ascii = False, indent = 4)
    print(f"Data saved to {file_name}")

save_data_to_json(all_store_info)

Data saved to store_info_db3.json
